In [6]:
import pandas as pd
import numpy as np
text=pd.read_csv("../input/resuts-image-and-text-100k/predictions_text 2.csv", header=None)
image=pd.read_csv("../input/resuts-image-and-text-100k/predictions_image.csv", header=None)
places=pd.read_csv("../input/test-instacities-clip/predictions_clip.csv")
test=pd.read_csv("../input/test-file-instacities/test.csv")
sum = image + text
max_i=sum.idxmax(axis="columns",skipna=False)
places["pred_city"]=max_i


In [7]:
#We make a selection with the predicted places with probability greater or equal than 0.6 and only the cases than the prediction of the city is succeded
pred_places_city_ok=places[places["probability"]>=0.6][places["city_id"]==places["pred_city"]]
pred_places_city=places[places["probability"]>=0.6]

len(pred_places_city_ok)/len(pred_places_city)

In [3]:
items_groupby_taxonomy=pred_places_city[pred_places_city["probability"]>=0.6].groupby(['tourism_taxonomy']).count()

In [4]:
items_groupby_taxonomy_ok=pred_places_city_ok[pred_places_city_ok["probability"]>=0.6].groupby(['tourism_taxonomy']).count()

In [5]:
items_groupby_taxonomy

In [14]:
items_groupby_taxonomy["ok"]=items_groupby_taxonomy_ok["id"]

In [18]:
#del items_groupby_taxonomy["probability"]
del items_groupby_taxonomy['city_id']
del items_groupby_taxonomy['pred_city']

In [22]:
items_groupby_taxonomy["acc"]=items_groupby_taxonomy["ok"]/items_groupby_taxonomy["id"]

In [26]:
items_groupby_taxonomy[items_groupby_taxonomy["id"]>10].sort_values(["acc"],ascending=False).head(20)

In [6]:
len(pred_places_city)

In [29]:
pred_places_city[pred_places_city["city_name"]=="sanfrancisco"][pred_places_city["tourism_taxonomy"]=="a bridge\n"]

In [45]:
test.loc[test["id"]==1487536655233955940]["text"][1017]


In [26]:
items_groupby_taxonomy_ok=predicted_places_with_predicted_city[predicted_places_with_predicted_city["probability"]>=0.6].groupby(['city_name','tourism_taxonomy']).count()

In [ ]:
predicted_places=places[places["probability"]>=0.6]
items_groupby_taxonomy=places[places["probability"]>=0.6].groupby(['city_name','tourism_taxonomy']).count()

In [5]:
del items_groupby_taxonomy['probability']
del items_groupby_taxonomy['city_id']
del items_groupby_taxonomy['pred_city']

In [6]:
items_groupby_taxonomy

In [28]:

items_groupby_taxonomy[items_groupby_taxonomy['id']>9]




In [ ]:
items_groupby_taxonomy.sort_values().tail(20)

In [4]:
labels=['a skyscraper\n','a clothing store\n','an art gallery\n','a beauty salon\n','a hospital\n','a shower\n','a crosswalk\n','a gift shop\n','a bakery shop\n', 'a bridge\n']
df = pd.DataFrame()
for i in labels:
    df[i]=places[places["probability"]>=0.6][places["tourism_taxonomy"]==i].groupby(['city_name'])["id"].count()
df=df.fillna(0)
for i in labels:
    df[i] = df[i].astype(int)


df

In [42]:
def to_na(x):
    if x>=10:
        return x
    else:
        return np.nan
#obtaining accuracies
labels=['a skyscraper\n','a clothing store\n','an art gallery\n','a beauty salon\n','a hospital\n','a shower\n','a crosswalk\n','a gift shop\n','a bakery shop\n', 'a bridge\n']
df = pd.DataFrame()
for i in labels:
    df_all = places[places["probability"]>=0.6][places["tourism_taxonomy"]==i].groupby(['city_name'])["id"].count()
    df_ok = places[places["probability"]>=0.6][places["tourism_taxonomy"]==i][places["city_id"]==places["pred_city"]].groupby(['city_name'])["id"].count()
    #df_all = df_all.astype(float)
    #df_ok = df_ok.astype(float)
    df[i]=df_ok.apply(to_na) / df_all.apply(to_na)
    #df[i] =df[i].astype(float)
#df=df.fillna(0)
#for i in labels:
#    df[i] = df[i].astype(int)


df

In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(df, annot=True,cmap="coolwarm", linewidths=.5, ax=ax, vmin=0, vmax=1)

In [17]:
ax

In [20]:
df=df.fillna(0)


df

In [15]:

for i in labels:
    df[i]=places[places["probability"]>=0.6][places["tourism_taxonomy"]==i].groupby(['city_name'])["id"].count().sort_values(ascending=False))
    

In [ ]:
places[places["probability"]>=0.6][places["tourism_taxonomy"]=="a bridge\n"].groupby(['city_name'])["id"].count().sort_values(ascending=False)

In [ ]:
places[places["probability"]>=0.6][places["tourism_taxonomy"]=="a plaza\n"].groupby(['city_name'])["id"].count().sort_values(ascending=False)

In [ ]:
places[places["probability"]>=0.6][places["tourism_taxonomy"]=="a skyscraper\n"].groupby(['city_name'])["id"].count().sort_values(ascending=False)

In [3]:
cities=places.groupby(['city_name'])['city_name'].count().index

In [7]:
df2 = pd.DataFrame()
for i in cities:
    df2[i]=places[places["probability"]>=0.6][places['city_name']==i].groupby(["tourism_taxonomy"])["id"].count().sort_values(ascending=False).head(5).index
df2.transpose()
    #df2.to_csv("kk.csv")

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(kk)

In [ ]:
total=places[places["probability"]>=0.6]

In [ ]:
pp.ProfileReport(total)

In [ ]:
places[places["city_id"]==places["pred_city"]]

In [ ]:
import glob
import os
import numpy as np

from sklearn import metrics, preprocessing, model_selection

import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

INPUT_PATH = "../input/instacities1m/"
IMAGE_PATH = "../input/instacities1m/InstaCities1M/img_resized_1M/cities_instagram"
MODEL_PATH = "../working/"
dfx = pd.read_csv(INPUT_PATH + "test.csv")
dfx["path"] = dfx["category"].astype(str) + "/" + dfx["id"].astype(str) + ".jpg"
    
lbl_enc = preprocessing.LabelEncoder()
dfx.category = lbl_enc.fit_transform(dfx.category.values)

test_image_paths = [os.path.join(IMAGE_PATH, x ) for x in dfx.path.values]
test_targets = dfx.category.values

In [ ]:
results_image = pd.read_csv("../input/resuts-image-and-text-100k/predictions_image.csv", header=None)
results_text = pd.read_csv("../input/resuts-image-and-text-100k/predictions_text 2.csv", header=None)

In [ ]:
def plot_image(img_dict):
    image_tensor = img_dict["image"]
    target = img_dict["targets"]
    print(target)
    plt.figure(figsize=(10, 10))
    image = image_tensor.permute(1, 2, 0) / 255
    plt.imshow(image)

In [ ]:
# code for displaying multiple images in one figure

#import libraries
import cv2
from matplotlib import pyplot as plt
def plot_images(im1,t1,im2,t2,im3,t3,im4,t4):
    # create figure
    fig = plt.figure(figsize=(10, 7))

    # setting values to rows and column variables
    rows = 2
    columns = 2

    # reading images
    Image1 = cv2.imread(im1)
    Image2 = cv2.imread(im2)
    Image3 = cv2.imread(im3)
    Image4 = cv2.imread(im4)

    # Adds a subplot at the 1st position
    fig.add_subplot(rows, columns, 1)

    # showing image
    plt.imshow(Image1)
    plt.axis('off')
    plt.title("Image 1: "+ t1)

    # Adds a subplot at the 2nd position
    fig.add_subplot(rows, columns, 2)

    # showing image
    plt.imshow(Image2)
    plt.axis('off')
    plt.title("Image 2: "+t2)

    # Adds a subplot at the 3rd position
    fig.add_subplot(rows, columns, 3)

    # showing image
    plt.imshow(Image3)
    plt.axis('off')
    plt.title("Image 3: "+t3)
    #plt.text(2, 0, "dsdsds")

    # Adds a subplot at the 4th position
    fig.add_subplot(rows, columns, 4)

    # showing image
    plt.imshow(Image4)
    plt.axis('off')
    plt.title("Image 4: "+t4)



In [ ]:
# code for displaying multiple images in one figure

#import libraries
import cv2
from matplotlib import pyplot as plt
def plot_images(im1,t1,im2,t2):
    # create figure
    fig = plt.figure(figsize=(10, 7))

    # setting values to rows and column variables
    rows = 1
    columns = 2

    # reading images
    Image1 = cv2.imread(im1)
    Image2 = cv2.imread(im2)


    # Adds a subplot at the 1st position
    fig.add_subplot(rows, columns, 1)

    # showing image
    plt.imshow(Image1)
    plt.axis('off')
    plt.title("Image 5: "+ t1)

    # Adds a subplot at the 2nd position
    fig.add_subplot(rows, columns, 2)

    # showing image
    plt.imshow(Image2)
    plt.axis('off')
    plt.title("Image 6: "+t2)

In [ ]:
results_text_n=pd.DataFrame(results_text_n)
results_image_n=pd.DataFrame(results_image_n)
suma=results_text_n+results_image_n

In [ ]:
plot_images(test_image_paths[1],lbl_enc.classes_[dfx.category[1]],test_image_paths[2],lbl_enc.classes_[dfx.category[2]],test_image_paths[0],lbl_enc.classes_[dfx.category[0]],test_image_paths[17],lbl_enc.classes_[dfx.category[17]])

In [ ]:
#max_t=results_text_n.max(axis="columns",skipna=False)
max_i=results_image.idxmax(axis="columns",skipna=False)
t=(max_i==dfx.category.values)
t[21:40]
#max_t=results_text.idxmax(axis="columns",skipna=False)
#ii=(max_t==dfx.category.values)
#ii[0:20]

In [ ]:
plot_images(test_image_paths[5],lbl_enc.classes_[dfx.category[5]],test_image_paths[34],lbl_enc.classes_[dfx.category[34]])

In [ ]:
print(lbl_enc.classes_[max_i[1]])
print(lbl_enc.classes_[max_i[2]])
print(lbl_enc.classes_[max_i[0]])
print(lbl_enc.classes_[max_i[17]])


In [ ]:
print(dfx.text[2][0:512]+"...")

In [ ]:
print(metrics.top_k_accuracy_score(dfx.category.values, suma,k=3))

In [ ]:
results_text_n.iloc[3].array

In [ ]:

k=np.vstack([results_text_n.iloc[3].array])
k=np.vstack([k,results_text_n.iloc[3].array])
k

In [ ]:
max_t=results_text_n.max(axis="columns",skipna=False)
max_i=results_image_n.max(axis="columns",skipna=False)


for i in range(0,100000):
    if max_t[i]>max_i[i]:
        if i==0:
            r=np.vstack([results_text_n.iloc[i].array])
        else:
            r=np.vstack([r,results_text_n.iloc[i].array])
    else:
        if i==0:
            r=np.vstack([results_image_n.iloc[i].array])
        else:
            r=np.vstack([r,results_image_n.iloc[i].array])
        

In [ ]:
r_p=pd.DataFrame(r)

In [ ]:
print(metrics.top_k_accuracy_score(dfx.category.values, r_p,k=3))

In [ ]:
results_text_n=preprocessing.normalize(results_text, norm="l1")
results_image_n=preprocessing.normalize(results_image, norm="l1")

In [ ]:
for i, j in zip(range(10), range(10)):
    print (i)
    print(j)

In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [ ]:
len(outputs.text_embeds.detach().cpu().tolist()[0])

In [ ]:
image_features.detach().numpy()[0]

In [ ]:
images_features[1]